# Return과 Yield

`return`은 일반 함수, `yield`는 반복 가능한 값을 순차적으로 생성할 때 사용.

- `return`은 함수를 즉시 종료하고 값을 반환  
- `yield`는 값을 하나 반환한 뒤 함수의 상태를 유지하며, 다음 호출 시 이어서 실행 - 제너레이터(generator)를 만들어 반복적으로 값을 생성.  


## 1. Return을 사용한 함수 (전체 데이터 로드)

파일 전체를 한 번에 읽어서 리스트로 반환하는 함수입니다.

In [ ]:
import csv

def read_csv_with_return(filename):
    """
    return을 사용하여 파일 전체를 한 번에 읽어 리스트로 반환
    메모리에 모든 데이터를 한 번에 로드
    """
    result = []
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            result.append(row)
    return result  # 함수 종료 시 모든 데이터를 한 번에 반환

# 사용 예제
print("=== Return을 사용한 방식 ===")
data = read_csv_with_return('05-sample.csv')
print(f"총 {len(data)}개의 행을 로드했습니다.")
print(f"타입: {type(data)}")
print("\n처음 3개 행:")
for i, row in enumerate(data[:3]):
    print(f"{i+1}. {row}")

## 2. Yield를 사용한 제너레이터 (순차적 처리)

파일을 한 줄씩 읽어서 순차적으로 반환하는 제너레이터 함수입니다.

In [ ]:
def read_csv_with_yield(filename):
    """
    yield를 사용하여 파일을 한 줄씩 읽어 순차적으로 반환
    메모리에 한 번에 하나의 행만 로드 (메모리 효율적)
    """
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            yield row  # 한 행을 반환하고 함수 상태를 유지

# 사용 예제
print("=== Yield를 사용한 방식 (제너레이터) ===")
data_generator = read_csv_with_yield('05-sample.csv')
print(f"타입: {type(data_generator)}")
print(f"제너레이터 객체: {data_generator}")

print("\n처음 3개 행 (next() 사용):")
print(f"1. {next(data_generator)}")
print(f"2. {next(data_generator)}")
print(f"3. {next(data_generator)}")

## 3. 제너레이터를 반복문으로 사용하기

제너레이터는 `for` 루프에서 자동으로 순회할 수 있습니다.

In [ ]:
print("=== 제너레이터를 for 루프로 사용 ===")
data_generator = read_csv_with_yield('05-sample.csv')

# 처음 5개만 출력
for i, row in enumerate(data_generator):
    print(f"{i+1}. {row['name']} ({row['age']}세, {row['city']}) - {row['salary']}원")
    if i >= 4:  # 5개만 출력
        break

print("\n※ 제너레이터는 한 번 순회하면 소진됩니다!")

## 4. 실용 예제: 특정 조건 필터링

급여가 4,000,000원 이상인 직원만 찾는 경우를 비교해봅시다.

In [ ]:
# Return 방식: 전체 데이터를 먼저 로드한 후 필터링
def find_high_salary_with_return(filename, min_salary):
    all_data = read_csv_with_return(filename)  # 전체 데이터 로드
    result = []
    for row in all_data:
        if int(row['salary']) >= min_salary:
            result.append(row)
    return result

# Yield 방식: 조건에 맞는 데이터만 순차적으로 반환
def find_high_salary_with_yield(filename, min_salary):
    for row in read_csv_with_yield(filename):  # 한 줄씩 처리
        if int(row['salary']) >= min_salary:
            yield row

# 비교
print("=== Return 방식으로 급여 >= 4,000,000원 찾기 ===")
high_salary_return = find_high_salary_with_return('05-sample.csv', 4000000)
print(f"찾은 인원: {len(high_salary_return)}명")
for person in high_salary_return[:3]:
    print(f"  - {person['name']}: {person['salary']}원")

print("\n=== Yield 방식으로 급여 >= 4,000,000원 찾기 ===")
high_salary_yield = find_high_salary_with_yield('05-sample.csv', 4000000)
print(f"타입: {type(high_salary_yield)}")
print("처음 3명만 출력:")
for i, person in enumerate(high_salary_yield):
    print(f"  - {person['name']}: {person['salary']}원")
    if i >= 2:
        break

## 5. 메모리 사용량 비교

`sys.getsizeof()`를 사용하여 메모리 사용량을 비교해봅시다.

In [ ]:
import sys

# Return 방식 - 전체 데이터를 리스트로 저장
data_list = read_csv_with_return('05-sample.csv')
list_size = sys.getsizeof(data_list)
print(f"=== Return (리스트) ===")
print(f"메모리 사용량: {list_size:,} bytes")
print(f"데이터 개수: {len(data_list)}개")
print(f"평균 메모리/항목: {list_size / len(data_list):.2f} bytes")

# Yield 방식 - 제너레이터 객체만 저장
data_generator = read_csv_with_yield('05-sample.csv')
generator_size = sys.getsizeof(data_generator)
print(f"\n=== Yield (제너레이터) ===")
print(f"메모리 사용량: {generator_size:,} bytes")
print(f"타입: {type(data_generator)}")

print(f"\n📊 메모리 차이: {list_size / generator_size:.1f}배")
print("\n💡 결론:")
print("  - Return: 모든 데이터를 메모리에 한 번에 로드 (빠른 접근, 큰 메모리)")
print("  - Yield: 필요할 때마다 한 개씩 생성 (낮은 메모리, 순차 접근)")

## 6. 주요 차이점 정리

| 특징 | Return | Yield |
|------|--------|-------|
| **반환 방식** | 함수 종료 시 모든 값을 한 번에 반환 | 값을 하나씩 반환하며 함수 상태 유지 |
| **메모리 사용** | 모든 데이터를 메모리에 저장 | 필요한 시점에 하나씩 생성 |
| **재사용** | 여러 번 사용 가능 (리스트) | 한 번 순회하면 소진됨 |
| **속도** | 초기 로딩 시간 있음, 이후 빠른 접근 | 필요할 때만 생성하므로 초기 지연 없음 |
| **적합한 경우** | 데이터가 작고, 전체 접근 필요 | 데이터가 크고, 순차 처리 충분 |
| **반환 타입** | `list`, `dict` 등 | `generator` 객체 |